### Import libraries and packages / Define needed functions / Call the whole path of required csv files

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytz
import os

from scipy.stats import boxcox, norm
from scipy.stats import laplace
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import beta
from scipy.stats import norm
from scipy.special import gamma
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


TRAINING_PATH = os.path.join(os.getcwd(), "1-1. train_y.csv")
SUBMISSION_PATH = os.path.join(os.getcwd(), "2. submission_format.csv")

GJ_LDAPS_TRAIN_PATH = os.path.join(os.getcwd(), "1-4. train_ldaps_gyeongju.pkl")
GJ_LDAPS_TEST_PATH = os.path.join(os.getcwd(), "1-4. test_ldaps_gyeongju.pkl")

YG_LDAPS_TRAIN_PATH = os.path.join(os.getcwd(), "1-3. train_ldaps_yeonggwang.pkl")
YG_LDAPS_TEST_PATH = os.path.join(os.getcwd(), "1-3. test_ldaps_yeonggwang.pkl")

GJ_SCADA_2020_PATH = os.path.join(os.getcwd(), "dynamic_report_ewp02_2020_10min.xlsx")
GJ_SCADA_2021_PATH = os.path.join(os.getcwd(), "dynamic_report_ewp02_2021_10min.xlsx")
GJ_SCADA_2022_PATH = os.path.join(os.getcwd(), "dynamic_report_ewp02_2022_10min.xlsx")

YG_SCADA_2007_PATH = os.path.join(os.getcwd(),"dynamic_report_ewp004_202001_202007.xlsx")
YG_SCADA_2012_PATH = os.path.join(os.getcwd(),"dynamic_report_ewp004_202008_202012.xlsx")
YG_SCADA_2106_PATH = os.path.join(os.getcwd(),"dynamic_report_ewp004_202101_202106.xlsx")
YG_SCADA_2112_PATH = os.path.join(os.getcwd(),"dynamic_report_ewp004_202107_202112.xlsx")
YG_SCADA_2206_PATH = os.path.join(os.getcwd(),"dynamic_report_ewp004_202201_202206.xlsx")
YG_SCADA_2212_PATH = os.path.join(os.getcwd(),"dynamic_report_ewp004_202207_202212.xlsx")


# function to Convert u, v vector to wind speed and direction.
def uv_to_wsd(u_wind_speed, v_wind_speed):
    u_ws = u_wind_speed.to_numpy()
    v_ws = v_wind_speed.to_numpy()

    # NOTE: http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv
    wind_speed = np.nansum([u_ws**2, v_ws**2], axis=0)**(1/2.)

    # math degree
    wind_direction = np.rad2deg(np.arctan2(v_ws, u_ws+1e-6))
    wind_direction[wind_direction < 0] += 360

    # meteorological degree
    wind_direction = 270 - wind_direction
    wind_direction[wind_direction < 0] += 360

    return wind_speed, wind_direction

### Manipulate the data into required form for analysis

In [16]:
def ldaps_manipulate(data):
    data_return = data
    data_return["wind_speed"], data_return["wind_direction"] = uv_to_wsd(data_return["wind_u_10m"], data_return["wind_v_10m"])
    data_return['density']=data_return['pressure']*(0.029)/(8.314*data_return['temp_air'])
    data_return['without_c']=(1/8000)*data_return['density']*np.pi*113*113*(data_return['wind_speed']**3)
    data_return['Date']=pd.to_datetime(data_return.index)
    
    return data_return

def train_y_manipulate(data, train_y, train_date, windfarm): # name of the windfarm 영광풍력 or 경주풍력
    data_return = data
    data_return=train_y[train_y['plant_name'] == windfarm]
    data_return['end_datetime']=pd.to_datetime(data_return['end_datetime'])
    data_return=data_return[data_return['end_datetime'].isin(train_date)]

    return data_return    

#### Manipulate train_data, test_data, match data for Yeonggwang

In [17]:
# Loading Yeonggwang data
train_y = pd.read_csv(TRAINING_PATH)

yg_train_x = pd.read_pickle(YG_LDAPS_TRAIN_PATH)
yg_train_x = ldaps_manipulate(yg_train_x)
yg_train_date=yg_train_x[yg_train_x['turbine_id']=='WTG01'].index

yg_test = pd.read_pickle(YG_LDAPS_TEST_PATH)
yg_test = ldaps_manipulate(yg_test)

yg_train_y= pd.read_csv(TRAINING_PATH)
yg_train_y = train_y_manipulate(yg_train_y, train_y, yg_train_date, "영광풍력") 

# Skip the process of manipulating and writing the file
# Instead just read the file that was written
YG_MATCH_PATH = os.path.join(os.getcwd(), "yg_match_data.csv")
yg_match_data = pd.read_csv(YG_MATCH_PATH)

"""
all_data = []

for i in range(35):
    check201 = pd.read_excel(YG_SCADA_2007_PATH,  sheet_name=i, skiprows=5)
    check202 = pd.read_excel(YG_SCADA_2012_PATH,  sheet_name=i, skiprows=5)
    check211 = pd.read_excel(YG_SCADA_2106_PATH,  sheet_name=i, skiprows=5)
    check212 = pd.read_excel(YG_SCADA_2112_PATH,  sheet_name=i, skiprows=5)
    check221 = pd.read_excel(YG_SCADA_2206_PATH,  sheet_name=i, skiprows=5)
    check222 = pd.read_excel(YG_SCADA_2212_PATH,  sheet_name=i, skiprows=5)

    check201 = check201[check201["WTG. Name"]==f"WTG{i+1:02}"]
    check202 = check202[check202["WTG. Name"]==f"WTG{i+1:02}"]
    check211 = check211[check211["WTG. Name"]==f"WTG{i+1:02}"]
    check212 = check212[check212["WTG. Name"]==f"WTG{i+1:02}"]
    check221 = check221[check221["WTG. Name"]==f"WTG{i+1:02}"]
    check222 = check222[check222["WTG. Name"]==f"WTG{i+1:02}"]

    all_data.extend([check201, check202, check211, check212, check221, check222])

yg_concat = pd.concat(all_data, ignore_index=True)

yg_concat["Date/Time"] = pd.to_datetime(yg_concat["Date/Time"], errors="coerce")

yg_concat.rename(columns={"Date/Time": "Date"}, inplace=True)
yg_concat.rename(columns={"WTG. Name": "turbine_id"},inplace=True)
yg_concat.rename(columns={"Rotor\nPitch 1 Angle\n[deg]": "state"},inplace =True )

yg_concat["Date"] = pd.to_datetime(yg_concat["Date"], format="%Y.%m.%d %H:%M", errors="coerce")
yg_concat["Date"] = yg_concat["Date"].dt.tz_localize(pytz.FixedOffset(540)) # Localize to timezone +09:00


yg_match_data = pd.merge(yg_concat, yg_train_x, on=["Date", "turbine_id"], how="right")
yg_match_data.to_csv("yg_match_data.csv",index=False)
"""

/var/folders/tf/02_6lkvd73nbp9k5fyfj80fh0000gn/T/ipykernel_32159/1479586426.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_return['end_datetime']=pd.to_datetime(data_return['end_datetime'])


'\nall_data = []\n\nfor i in range(35):\n    check201 = pd.read_excel(YG_SCADA_2007_PATH,  sheet_name=i, skiprows=5)\n    check202 = pd.read_excel(YG_SCADA_2012_PATH,  sheet_name=i, skiprows=5)\n    check211 = pd.read_excel(YG_SCADA_2106_PATH,  sheet_name=i, skiprows=5)\n    check212 = pd.read_excel(YG_SCADA_2112_PATH,  sheet_name=i, skiprows=5)\n    check221 = pd.read_excel(YG_SCADA_2206_PATH,  sheet_name=i, skiprows=5)\n    check222 = pd.read_excel(YG_SCADA_2212_PATH,  sheet_name=i, skiprows=5)\n\n    check201 = check201[check201["WTG. Name"]==f"WTG{i+1:02}"]\n    check202 = check202[check202["WTG. Name"]==f"WTG{i+1:02}"]\n    check211 = check211[check211["WTG. Name"]==f"WTG{i+1:02}"]\n    check212 = check212[check212["WTG. Name"]==f"WTG{i+1:02}"]\n    check221 = check221[check221["WTG. Name"]==f"WTG{i+1:02}"]\n    check222 = check222[check222["WTG. Name"]==f"WTG{i+1:02}"]\n\n    all_data.extend([check201, check202, check211, check212, check221, check222])\n\nyg_concat = pd.concat(al

#### Manipulate train_data, test_data, match data for Gyeongju

In [18]:
# Loading Gyeongju data
train_y = pd.read_csv(TRAINING_PATH)

gj_train_x = pd.read_pickle(GJ_LDAPS_TRAIN_PATH)
gj_train_x = ldaps_manipulate(gj_train_x)
gj_train_date=gj_train_x[gj_train_x['turbine_id']=='WTG01'].index

gj_test = pd.read_pickle(GJ_LDAPS_TEST_PATH)
gj_test = ldaps_manipulate(gj_test)

gj_train_y= pd.read_csv(TRAINING_PATH)
gj_train_y = train_y_manipulate(gj_train_y, train_y, gj_train_date, "경주풍력") 

# Skip the process of manipulating and writing the file
# Instead just read the file that was written
GJ_MATCH_PATH = os.path.join(os.getcwd(), "gj_match_data.csv")
gj_match_data = pd.read_csv(GJ_MATCH_PATH)

"""
all_data2 = []

for i in range(9):
    check201 = pd.read_excel(GJ_SCADA_2020_PATH,  sheet_name=i, skiprows=5)
    check211 = pd.read_excel(GJ_SCADA_2021_PATH,  sheet_name=i, skiprows=5)
    check221 = pd.read_excel(GJ_SCADA_2022_PATH,  sheet_name=i, skiprows=5)

    check201 = check201[check201["WTG. Name"]==f"WTG{i+1:02}"]
    check211 = check211[check211["WTG. Name"]==f"WTG{i+1:02}"]
    check221 = check221[check221["WTG. Name"]==f"WTG{i+1:02}"]

    all_data2.extend([check201, check211, check221])

gj_concat = pd.concat(all_data2, ignore_index=True)

gj_concat["Date/Time"] = pd.to_datetime(gj_concat["Date/Time"], errors="coerce")

gj_concat.rename(columns={"Date/Time": "Date"}, inplace=True)
gj_concat.rename(columns={"WTG. Name": "turbine_id"},inplace=True)
gj_concat.rename(columns={"Rotor\nPitch 1 Angle\n[deg]": "state"},inplace =True )

gj_concat["Date"] = pd.to_datetime(gj_concat["Date"], format="%Y.%m.%d %H:%M", errors="coerce")
gj_concat["Date"] = gj_concat["Date"].dt.tz_localize(pytz.FixedOffset(540)) # Localize to timezone +09:00

gj_match_data = pd.merge(gj_concat, gj_train_x, on=["Date", "turbine_id"], how="right")
gj_match_data.to_csv("gj_match_data.csv",index=False)
"""

/var/folders/tf/02_6lkvd73nbp9k5fyfj80fh0000gn/T/ipykernel_32159/1479586426.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_return['end_datetime']=pd.to_datetime(data_return['end_datetime'])


'\nall_data2 = []\n\nfor i in range(9):\n    check201 = pd.read_excel(GJ_SCADA_2020_PATH,  sheet_name=i, skiprows=5)\n    check211 = pd.read_excel(GJ_SCADA_2021_PATH,  sheet_name=i, skiprows=5)\n    check221 = pd.read_excel(GJ_SCADA_2022_PATH,  sheet_name=i, skiprows=5)\n\n    check201 = check201[check201["WTG. Name"]==f"WTG{i+1:02}"]\n    check211 = check211[check211["WTG. Name"]==f"WTG{i+1:02}"]\n    check221 = check221[check221["WTG. Name"]==f"WTG{i+1:02}"]\n\n    all_data2.extend([check201, check211, check221])\n\ngj_concat = pd.concat(all_data2, ignore_index=True)\n\ngj_concat["Date/Time"] = pd.to_datetime(gj_concat["Date/Time"], errors="coerce")\n\ngj_concat.rename(columns={"Date/Time": "Date"}, inplace=True)\ngj_concat.rename(columns={"WTG. Name": "turbine_id"},inplace=True)\ngj_concat.rename(columns={"Rotor\nPitch 1 Angle\n[deg]": "state"},inplace =True )\n\ngj_concat["Date"] = pd.to_datetime(gj_concat["Date"], format="%Y.%m.%d %H:%M", errors="coerce")\ngj_concat["Date"] = gj_c

### Make a model for Analysis

In [19]:
# defining beta function that will be used on the prediction model
def beta_function(alpha, beta):
    return gamma(alpha) * gamma(beta) / gamma(alpha + beta)

In [24]:
# THE PREDICTION MODEL
def forecasting(location):
    if location == "영광풍력":
        locat = "영광풍력"
        match_data = yg_match_data
        train_data = yg_train_y
        test_data = yg_test

    else:
        locat = "경주풍력"
        match_data = gj_match_data
        train_data = gj_train_y
        test_data = gj_test

    binary_data = match_data[-match_data['state'].isna()]
    binary_data['binary_state'] = 1*(binary_data['state']>=45)


    print("@@@@@ Transfoming Data @@@@@@")

    transformed_data, lambd=boxcox(np.array(binary_data[binary_data['binary_state']==0]['wind_speed']))
    mean_val=np.mean(transformed_data)
    var_val=np.var(transformed_data)
      
    q=sum(binary_data['binary_state']==0)/len(binary_data)
    
    transformed_data2, lambd2=boxcox(np.array(binary_data['wind_speed']))
    mean_val2=np.mean(transformed_data2)
    var_val2=np.var(transformed_data2)
       
    #in test data

    test_trans_data=(test_data['wind_speed']**lambd-1)/lambd
    p=norm.pdf(test_trans_data,loc=mean_val, scale=np.sqrt(var_val))
    test_trans_data2=(test_data['wind_speed']**lambd2-1)/lambd2
    r=norm.pdf(test_trans_data2,loc=mean_val2,scale=np.sqrt(var_val2))
    
    test_data['proba_0']=p*q/r
    test_data['no_c_with_proba']=test_data['proba_0']*test_data['without_c']


    print("@@@@@ Scaling Data @@@@@@")

    #diff scaling
    real_diff=train_data['energy_kwh'].diff(1)[1:]
    test_diff = test_data.groupby('Date')['no_c_with_proba'].sum().diff(1)[1:]

    loc2, scale2 = laplace.fit(real_diff)
    loc3, scale3 = laplace.fit(test_diff)

    test_diffs=loc2+(test_diff-loc3)*(scale2/scale3)
    scaled_test_diff=test_diffs-np.mean(test_diff)
    adjust_test_cumsum=np.cumsum(np.insert(scaled_test_diff,0,np.array(test_data.groupby('Date')['no_c_with_proba'].sum())[0]))


    #beta-dist scaling
    scaler = MinMaxScaler(feature_range=(0,1))

    data2_scaled=scaler.fit_transform(np.array(train_data['energy_kwh']).reshape(-1,1)).flatten()
    data2_scaled=np.clip(data2_scaled,1e-6,1-(1e-6))
    a2,b2,loc2,scale2=beta.fit(data2_scaled,floc=0,fscale=1)

    test_scaled=scaler.fit_transform(adjust_test_cumsum.reshape(-1,1)).flatten()
    test_scaled=np.clip(test_scaled,1e-6,1-(1e-6))
    a3,b3,loc3,scale3=beta.fit(test_scaled,floc=0,fscale=1)


    test_cdf=beta.cdf(test_scaled,a3,b3)
    y=beta.ppf(test_cdf,a2,b2)

    x=y*(max(train_data['energy_kwh'])-min(train_data['energy_kwh']))+min(train_data['energy_kwh'])

    pred_y=pd.DataFrame({
    "predict_energy_kwh":x
    })
    pred_y.index=test_data[test_data['turbine_id']=='WTG01'].index
    # pred_y.index=train_data['end_datetime']
    pred_y.index.name='dt'

    print("Prediction for", locat, "is done")
    return(pred_y)


### Excute the Model

In [25]:
pred_yg=forecasting("영광풍력")

/var/folders/tf/02_6lkvd73nbp9k5fyfj80fh0000gn/T/ipykernel_32159/3559179510.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_data['binary_state'] = 1*(binary_data['state']>=45)


@@@@@ Transfoming Data @@@@@@
@@@@@ Scaling Data @@@@@@
Prediction for 영광풍력 is done


In [26]:
pred_gj=forecasting("경주풍력")

/var/folders/tf/02_6lkvd73nbp9k5fyfj80fh0000gn/T/ipykernel_32159/3559179510.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_data['binary_state'] = 1*(binary_data['state']>=45)


@@@@@ Transfoming Data @@@@@@
@@@@@ Scaling Data @@@@@@
Prediction for 경주풍력 is done


In [29]:
submission=pd.read_csv(SUBMISSION_PATH,encoding='euc-kr')
submission[submission['plant_name']=='경주풍력']['energy_kwh']=np.array(pred_gj['predict_energy_kwh'])
submission[submission['plant_name']=='영광풍력']['energy_kwh']=np.array(pred_yg['predict_energy_kwh'])
submission['energy_kwh']=(pd.concat([pred_gj['predict_energy_kwh'],pred_yg['predict_energy_kwh']])).values

/var/folders/tf/02_6lkvd73nbp9k5fyfj80fh0000gn/T/ipykernel_32159/3645417919.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission[submission['plant_name']=='경주풍력']['energy_kwh']=np.array(pred_gj['predict_energy_kwh'])
/var/folders/tf/02_6lkvd73nbp9k5fyfj80fh0000gn/T/ipykernel_32159/3645417919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission[submission['plant_name']=='영광풍력']['energy_kwh']=np.array(pred_yg['predict_energy_kwh'])


In [32]:
submission.to_csv('0. submission_data.csv')